<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/AdiabaticFlameTemperature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --depth 1 https://github.com/profteachkids/CHE2064.git &> /dev/null
!pip install DotMap &> /dev/null
import sys
sys.path.insert(1, "/content/CHE2064") #Path to CHE module imports

In [2]:
from dotmap import DotMap
import pandas as pd
import jax
from jax.lax import stop_gradient
import jax.numpy as jnp
from jax.config import config
config.update("jax_enable_x64", True) #JAX default is 32bit single precision
from tools.tree_array_transform2 import VSC, Comp, Range
import tools.che as che
R=8.314 # J/(mol K)

In [3]:
p = che.Props(['Oxygen', 'Nitrogen', 'CarbonDioxide', 'Methane', 'Water'])

/usr/local/lib/python3.6/dist-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [4]:
def model(c):
    r=DotMap()
    r.o2_feed = 2*c.ch4 * (1+c.excess_o2)
    r.n2 = r.o2_feed * 79/21
    r.co2_prod = c.ch4
    r.o2_prod = r.o2_feed - 2*c.ch4

    r.nfeed = jnp.array([r.o2_feed, r.n2, 0., c.ch4, 0.])
    r.nprod = jnp.array([r.o2_prod, r.n2, r.co2_prod, 0., 2*c.ch4])
    r.hfeed = p.HIG(r.nfeed, c.feedT)
    r.hprod = p.HIG(r.nprod, c.T)

    return  [r.hfeed - r.hprod], r

In [10]:
c=DotMap()
c.ch4 = 1.
c.excess_o2 = 0.
c.feedT = 300
c.T = Range(1000., 300., 5000.)

vsc = VSC(c, model)
vsc.solve()

[-497723.19380297]
[39104.30333949]
[-331.78617337]
[-0.02030654]
[-7.27595761e-11]


In [11]:
vsc.cdf

,vector1
,1
T,2328.340692
ch4,1.000000
excess_o2,0.000000
feedT,300.000000


In [9]:
vsc.rdf

vector1 vector5                  
                1       1        2  3  4  5
o2_feed       2.4                          
n2        9.02857                          
co2_prod        1                          
o2_prod       0.4                          
nfeed                 2.4  9.02857  0  1  0
nprod                 0.4  9.02857  1  0  2
hfeed    -74167.1                          
hprod    -74167.1

In [15]:
vsc.v.T-298.15

DeviceArray(2030.19069164, dtype=float64)

In [16]:
def model2(c):
    r=DotMap()
    r.o2_feed = 2*c.ch4 * (1+c.excess_o2)
    r.n2 = r.o2_feed * 79/21
    r.co2_prod = c.ch4
    r.o2_prod = r.o2_feed - 2*c.ch4

    r.nfeed = jnp.array([r.o2_feed, r.n2, 0., c.ch4, 0.])
    r.nprod = jnp.array([r.o2_prod, r.n2, r.co2_prod, 0., 2*c.ch4])
    r.hfeed = p.HIG(r.nfeed, c.feedT)
    r.hprod = p.HIG(r.nprod, c.T)

    return  [r.hfeed - r.hprod], r

In [17]:
c=DotMap()
c.ch4 = 1.
c.feedT = 300
c.T = 2000.
c.excess_o2 = Range(0.2, 0., 1.)

vsc = VSC(c, model)
vsc.solve()

[-4385.7877102]
[-40.66902105]
[-0.00366965]
[4.36557457e-11]


In [18]:
vsc.vdf

,vector1
,1
excess_o2,0.265989


In [19]:
vsc.cdf

,vector1
,1
T,2000.000000
ch4,1.000000
excess_o2,0.265989
feedT,300.000000


In [20]:
vsc.rdf

vector1   vector5                  
                 1         1        2  3  4  5
o2_feed    2.53198                            
n2         9.52506                            
co2_prod         1                            
o2_prod   0.531978                            
nfeed                2.53198  9.52506  0  1  0
nprod               0.531978  9.52506  1  0  2
hfeed     -74133.2                            
hprod     -74133.2